In [1]:
import numpy as np
import pandas as pd
import sqlite3
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from json import loads, dumps

## Connecting to the database and reading in the data

In [2]:
con = sqlite3.connect('/Users/sumaiyanathani/Desktop/Bootcamp/project3/cancer.sqlite')
engine = create_engine("sqlite:////Users/sumaiyanathani/Desktop/Bootcamp/project3/cancer.sqlite")

In [19]:
#wb = pd.read_excel("/Users/sumaiyanathani/Downloads/2022_supplementary_online_data.xlsx", sheet_name = "SuppT3 - geographic")
#wb.to_sql('provincial_data',con,index=False)

In [3]:
data = pd.read_sql("SELECT * FROM provincial_data", con)

In [21]:
#Base = automap_base()

In [4]:
#Base.prepare(autoload_with=engine)
session = Session(engine)

## Cleaning the data

In [8]:
data = data.iloc[2:-4]
data.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
2,Province,Cancer_type,Both sexes 2 years,Males 2 years,Females 2 years,Total 5 years,Males 5 years,Females 5 years,Total 25 years,Males 25 years,Females 25 years
3,NL,All cancers,5500,2775,2725,11640,5835,5805,26905,13065,13840
4,NL,Lung and bronchus,530,260,270,880,425,455,1270,615,660
5,NL,Breast,825,5,820,1960,10,1950,5170,30,5140
6,NL,Colorectal,970,540,430,2055,1170,890,4675,2555,2120


In [9]:
provincial_data = data.rename(columns=data.iloc[0]).loc[3:]
provincial_data = provincial_data.loc[:, "Province": "Females 5 years"]
provincial_data.reset_index()

,index,Province,Cancer_type,Both sexes 2 years,Males 2 years,Females 2 years,Total 5 years,Males 5 years,Females 5 years
0,3,NL,All cancers,5500,2775,2725,11640,5835,5805
1,4,NL,Lung and bronchus,530,260,270,880,425,455
2,5,NL,Breast,825,5,820,1960,10,1950
3,6,NL,Colorectal,970,540,430,2055,1170,890
4,7,NL,Prostate,845,845,0,1940,1940,0
...,...,...,...,...,...,...,...,...,...
305,308,NU,Esophagus,0,0,0,0,0,0
306,309,NU,Cervix,5,0,0,10,0,10
307,310,NU,Testis,0,0,0,5,0,0
308,311,NU,Hodgkin lymphoma,0,0,0,0,0,0


In [10]:
provincial_data2 = provincial_data.dropna(axis = 0)
provincial_data2 = provincial_data2[provincial_data2.Province != "Province"]
provincial_data2 = provincial_data2.rename(columns={"Both sexes 2 years": "Both_sexes",
                                                    "Male 2 years":"Male_2_years",
                                                    "Females 2 years": "Females_2_years",
                                                    "Total 5 years": "Total_5_years",
                                                    "Males 5 years": "Males_5_years",
                                                    "Females 5 years": "Females_5_years"
                                                    })

In [11]:
provincial_data2

,Province,Cancer_type,Both_sexes,Males 2 years,Females_2_years,Total_5_years,Males_5_years,Females_5_years
3,NL,All cancers,5500,2775,2725,11640,5835,5805
4,NL,Lung and bronchus,530,260,270,880,425,455
5,NL,Breast,825,5,820,1960,10,1950
6,NL,Colorectal,970,540,430,2055,1170,890
7,NL,Prostate,845,845,0,1940,1940,0
...,...,...,...,...,...,...,...,...
308,NU,Esophagus,0,0,0,0,0,0
309,NU,Cervix,5,0,0,10,0,10
310,NU,Testis,0,0,0,5,0,0
311,NU,Hodgkin lymphoma,0,0,0,0,0,0


In [12]:
cancer_types = provincial_data2['Cancer_type'].unique()
cancer_types

array(['All cancers', 'Lung and bronchus', 'Breast', 'Colorectal',
       'Prostate', 'Bladder', 'Non-Hodgkin lymphoma', 'Melanoma',
       'Uterus (body, NOS)', 'Kidney and renal pelvis', 'Head and neck',
       'Pancreas', 'Leukemia', 'Thyroid', 'Stomach', 'Multiple myeloma',
       'Liver', 'Brain/CNS', 'Ovary', 'Esophagus', 'Cervix', 'Testis',
       'Hodgkin lymphoma', 'All other cancers'], dtype=object)

In [13]:
result = provincial_data2.to_json(orient="records")
parsed = loads(result)
json_object = dumps(parsed)  
json_object

'[{"Province": "NL", "Cancer_type": "All cancers", "Both_sexes": "5500", "Males 2 years": "2775", "Females_2_years": "2725", "Total_5_years": "11640", "Males_5_years": "5835", "Females_5_years": "5805"}, {"Province": "NL", "Cancer_type": "Lung and bronchus", "Both_sexes": "530", "Males 2 years": "260", "Females_2_years": "270", "Total_5_years": "880", "Males_5_years": "425", "Females_5_years": "455"}, {"Province": "NL", "Cancer_type": "Breast", "Both_sexes": "825", "Males 2 years": "5", "Females_2_years": "820", "Total_5_years": "1960", "Males_5_years": "10", "Females_5_years": "1950"}, {"Province": "NL", "Cancer_type": "Colorectal", "Both_sexes": "970", "Males 2 years": "540", "Females_2_years": "430", "Total_5_years": "2055", "Males_5_years": "1170", "Females_5_years": "890"}, {"Province": "NL", "Cancer_type": "Prostate", "Both_sexes": "845", "Males 2 years": "845", "Females_2_years": "0", "Total_5_years": "1940", "Males_5_years": "1940", "Females_5_years": "0"}, {"Province": "NL", "

In [14]:
with open("Resources/provincial_data.json", "w") as outfile:
    outfile.write(json_object)

In [28]:
con.commit()
con.close()